In [11]:
import re
import random
from mhcflurry import Class1AffinityPredictor
from tqdm import tqdm
import sys

In [12]:
predictor = Class1AffinityPredictor.load()

Forcing tensorflow backend.


2024-01-22 07:52:01.957442: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 07:52:01.981066: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-22 07:52:01.981535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 07:52:02.485599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-22 07:52:03.154759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-01-22 07:52:03.154791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information f

In [23]:
synth = pd.read_csv('results/synthetic_results.csv')

In [24]:
mhc_molecule_list = synth['mhc_molecule'].unique().tolist()
tcr_sequence_list = synth['tcr_sequence'].unique().tolist()

In [142]:
def tcr_reward(curr_seq, tcr_sequences):
    rewards = []
    for tcr_sequence in tcr_sequences:
        tcr_predict = tcrgp.predict([curr_seq], [tcr_sequences])
        predicted_tcr = tcr_predict[0]
        rewards.append(predicted_tcr)
    
    return rewards

def classification_reward(curr_seq):
    peptide_sequence = [curr_seq]  
    probabilities = classifier.predict_proba(peptide_sequence)
    class_1_probabilities = [prob[1] for prob in probabilities]
    return class_1_probabilities


# Load valid amino acids
def load_valid_aas(dictname):
    aas = []
    fh = open(dictname, 'r')
    for line in fh:
        spl = re.split('\s+', line.strip())
        aas.append(spl[0])
    fh.close()
    return aas


# Eliminate empty characters from a sequence
def eliminate_empty(seq):
    return seq.replace('0', '')

# Randomly select an amino acid
def random_aa(aas):
    return aas[random.randint(0, len(aas) - 1)]

# Generate a sequence within a length range
def generate_seq(lmin, lmax, aas):
    seq = ''
    ltarg = random.randint(lmin, lmax)
    while len(seq) < ltarg:
        seq += eliminate_empty(random_aa(aas))
    return seq

# Alter a sequence by replacing an amino acid
def alter_seq(seq, aas):
    ind = random.randint(0, len(seq))
    seq = list(seq)
    aa = random_aa(aas)
    if ind < len(seq):
        seq[ind] = aa
    else:
        seq.append(aa)
    seq = ''.join(seq)
    return eliminate_empty(seq)

# Calculate MHC binding affinity
def get_binding_affinity(peptide_sequence, mhc_proteins):
    rewards = []
    for mhc_protein in mhc_proteins:
        # Predict binding affinity using MHCflurry 2.0 model
        predicted_ic50s = predictor.predict([peptide_sequence], [mhc_protein])
        predicted_ic50 = predicted_ic50s[0]
        
        # Normalize the predicted IC50 to a reward value between 0 and 1
        max_ic50 = 50000
        reward = 1.0 - (predicted_ic50 / max_ic50)
        
        rewards.append(reward)
    
    return rewards

# Update selected sequences
def update_selected(selected, threshold_mhc, threshold_tcr, seq, mhc, rew_mhc, rew_tcr, rew_class, tcr_sequence):
    if rew_mhc > threshold_mhc and rew_tcr > threshold_tcr and rew_class == 1 and seq not in selected:
        selected[seq] = {'MHC': mhc, 'Rew_MHC': rew_mhc, 'TCR_Seq': tcr_sequence, 'Rew_TCR': rew_tcr, 'Rew_Class': rew_class}
        print(' selected', end='')
    return selected

In [25]:
import numpy as np

def needleman_wunsch_normalized(seq1, seq2, match=2, mismatch=1, gap=0):
    def calculate_max_score(seq1, seq2, match, mismatch, gap):
        # Calculate the maximum possible score
        max_score = max(len(seq1), len(seq2)) * match
        return max_score

#     Create a matrix to store alignment scores
    matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))

    #Initialize the first row and first column with gap penalties
    for i in range(len(seq1) + 1):
        matrix[i][0] = i * gap
    for j in range(len(seq2) + 1):
        matrix[0][j] = j * gap

    # Fill in the matrix
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            match_score = matrix[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch) 
            delete_score = matrix[i-1][j] + gap
            insert_score = matrix[i][j-1] + gap
            matrix[i][j] = max(match_score, delete_score, insert_score)

    # Calculate the normalized similarity score
    similarity_score = matrix[len(seq1)][len(seq2)] / calculate_max_score(seq1, seq2, match, mismatch, gap)

    return similarity_score

# Check sequence similarity using your Needleman-Wunsch algorithm
def is_sequence_similar_to_selected(new_seq, selected_seq, similarity_threshold):
    for selected_peptide in selected_seq:
        similarity_score = needleman_wunsch_normalized(new_seq, selected_peptide)  

        if similarity_score >= similarity_threshold:
            return True
    return False


lmin = 8
lmax = 12
thr_rew_mhc = 0.82  # Threshold for binding affinity
thr_rew_tcr = 0.53  # Threshold for TCR reward
similarity_threshold = 0.7


# Extend the list of MHC proteins and TCR sequences
mhc_proteins = mhc_molecule_list
tcr_sequences = tcr_sequence_list
aas = load_valid_aas('generators\Dictionary')

# Initialize selected sequences dictionary and output data list
selected_seq = {}
output_data = []

# Define a dictionary to track the count of each MHC molecule
mhc_counts = {mhc: 0 for mhc in mhc_proteins}
tcr_seq_counts = {tcr: 0 for tcr in tcr_sequences}

# Initialize max MHC and TCR values and corresponding indices
max_mhc_values = [-1] * len(mhc_proteins)
max_tcr_values = [-1] * len(tcr_sequences)
max_mhc_idx = -1
max_tcr_idx = -1

# Initialize progress bar
pbar = tqdm(desc="Processing sequences")

curr_seq = generate_seq(lmin, lmax, aas)  # Initialize curr_seq
curr_rew_mhc_values = get_binding_affinity(curr_seq, mhc_proteins) 
curr_rew_tcr = tcr_reward(curr_seq, tcr_sequences)
curr_rew_class = classification_reward(curr_seq)


while len(selected_seq) < 100:
    new_seq = alter_seq(curr_seq, aas)
    accept = False
    selected_in_iteration = False

    if new_seq != curr_seq and lmin <= len(new_seq) <= lmax:
        new_rew_class = classification_reward(new_seq)

        if new_rew_class > 0.5:  # Updated condition
            # Check if the new sequence is similar to any selected sequence
            if not is_sequence_similar_to_selected(new_seq, selected_seq.keys(), similarity_threshold):
                new_rew_mhc_values = get_binding_affinity(new_seq, mhc_proteins)
                new_rew_tcr_values = tcr_reward(new_seq, tcr_sequences)

                max_mhc_idx = new_rew_mhc_values.index(max(new_rew_mhc_values))
                max_tcr_idx = new_rew_tcr_values.index(max(new_rew_tcr_values))

                # Check if the selected MHC molecule and TCR sequence have exceeded the maximum count
                selected_mhc = mhc_proteins[max_mhc_idx]
                selected_tcr = tcr_sequences[max_tcr_idx]

                if mhc_counts[selected_mhc] < 5 and tcr_seq_counts[selected_tcr] < 5:
                    product_value = new_rew_mhc_values[max_mhc_idx] * new_rew_tcr_values[max_tcr_idx]

                    if product_value > (curr_rew_mhc_values[max_mhc_idx] * curr_rew_tcr[max_tcr_idx]) - 0.35 * random.random():
                        accept = True
                        if new_seq not in selected_seq:
                            selected_in_iteration = True
                            # Update the count of the selected MHC molecule and TCR sequence
                            mhc_counts[selected_mhc] += 1
                            tcr_seq_counts[selected_tcr] += 1
                            selected_seq = update_selected(selected_seq, thr_rew_mhc, thr_rew_tcr, new_seq, selected_mhc, new_rew_mhc_values[max_mhc_idx], new_rew_tcr_values[max_tcr_idx], curr_rew_class, selected_tcr)
                            pbar.update(1)

                # Initialize max MHC and TCR values and corresponding indices
                max_mhc_values = [-1] * len(mhc_proteins)
                max_tcr_values = [-1] * len(tcr_sequences)
                max_mhc_idx = -1
                max_tcr_idx = -1

                if selected_in_iteration:
                    data_dict = {
                        "Sequence": new_seq,
                        **{f"MHC_{i+1}": new_rew_mhc_values[i] for i in range(len(mhc_proteins))},
                        **{f"TCR_{i+1}": new_rew_tcr_values[i] for i in range(len(tcr_sequences))},
                        "Classification": new_rew_class,  # Add new_rew_class here
                    }
                    output_data.append(data_dict)  # Ensure data_dict is appended to output_data

                # Remove MHC molecule if its count reaches 10
                if mhc_counts[selected_mhc] >= 10:
                    mhc_proteins.remove(selected_mhc)

                # Remove TCR sequence if its count reaches 10
                if tcr_seq_counts[selected_tcr] >= 10:
                    tcr_sequences.remove(selected_tcr)

pbar.close()


Processing sequences: 0it [00:00, ?it/s]
Processing sequences: 1it [00:03,  3.18s/it]
Processing sequences: 2it [00:04,  2.25s/it]
Processing sequences: 3it [00:06,  1.95s/it]
Processing sequences: 4it [00:07,  1.80s/it]
Processing sequences: 5it [00:09,  1.72s/it]
Processing sequences: 6it [00:11,  1.67s/it]
Processing sequences: 7it [00:12,  1.65s/it]
Processing sequences: 8it [00:15,  2.13s/it]
Processing sequences: 9it [00:25,  4.44s/it]
Processing sequences: 10it [00:45,  9.43s/it]
Processing sequences: 11it [00:58, 10.51s/it]
Processing sequences: 12it [01:15, 12.35s/it]
Processing sequences: 13it [01:20, 10.09s/it]
Processing sequences: 14it [01:45, 14.48s/it]
Processing sequences: 15it [01:46, 10.61s/it]
Processing sequences: 16it [01:58, 10.87s/it]
Processing sequences: 17it [02:11, 11.64s/it]
Processing sequences: 18it [02:18, 10.11s/it]
Processing sequences: 19it [02:26,  9.54s/it]
Processing sequences: 20it [02:31,  8.14s/it]
Processing sequences: 21it [02:42,  9.11s/it]
P

KeyboardInterrupt: 

In [27]:
selected_seq 

{'KGESIICIIFY': {'MHC': 'B*14:02',
  'Rew_MHC': 0.998305738949775,
  'TCR_Seq': 'CASRYTTGDSYEQYF',
  'Rew_TCR': 0.9857495589472043,
  'Classification': 0.8952227205725236},
 'GEPWSWVLGMQ': {'MHC': 'A*02:01',
  'Rew_MHC': 0.9952393838380741,
  'TCR_Seq': 'CASSFDPGTGYFEQYF',
  'Rew_TCR': 0.9103326787856073,
  'Classification': 0.5175404946085987},
 'YVPEQSWKYS': {'MHC': 'A*80:01',
  'Rew_MHC': 0.9975548859986684,
  'TCR_Seq': 'CASSLVVGQPSYEQYF',
  'Rew_TCR': 0.9867157006058705,
  'Classification': 0.8447803123067259},
 'NDAYALGMDNK': {'MHC': 'C*06:02',
  'Rew_MHC': 0.9980542926363493,
  'TCR_Seq': 'CASRLQNVSPLHF',
  'Rew_TCR': 0.8585986534337591,
  'Classification': 0.7989730295128217},
 'MWKTHDHPTDY': {'MHC': 'B*39:01',
  'Rew_MHC': 0.9983832708365995,
  'TCR_Seq': 'CASSPPTSGIPEQYF',
  'Rew_TCR': 0.9540402012188997,
  'Classification': 0.7288610495952547},
 'YRGDRPGMNY': {'MHC': 'B*37:01',
  'Rew_MHC': 0.9978410271195682,
  'TCR_Seq': 'CASSFDPGTGYFEQYF',
  'Rew_TCR': 0.978165145823791,


In [29]:
mhc_counts

{'C*08:02': 4,
 'A*02:19': 2,
 'A*31:01': 3,
 'A*02:12': 3,
 'B*35:03': 2,
 'A*80:01': 2,
 'A*24:02': 2,
 'A*01:01': 3,
 'B*57:01': 3,
 'B*35:01': 4,
 'B*54:01': 1,
 'B*38:01': 2,
 'A*30:01': 3,
 'A*24:03': 5,
 'B*40:02': 2,
 'B*15:03': 1,
 'B*45:01': 4,
 'B*15:01': 2,
 'C*12:03': 3,
 'A*30:02': 1,
 'A*23:01': 2,
 'B*46:01': 2,
 'B*40:01': 2,
 'B*44:03': 3,
 'B*18:01': 3,
 'A*33:01': 3,
 'B*53:01': 1,
 'A*02:06': 1,
 'A*02:11': 2,
 'C*04:01': 2,
 'A*66:01': 2,
 'B*08:01': 1,
 'A*29:02': 2,
 'A*25:01': 1,
 'A*02:02': 1,
 'A*32:01': 1,
 'A*26:02': 2,
 'A*02:10': 1,
 'C*14:02': 3,
 'B*08:02': 2,
 'B*07:02': 1,
 'B*44:02': 1,
 'A*68:02': 1,
 'A*69:01': 2,
 'A*03:01': 1,
 'A*68:01': 2,
 'B*48:01': 1,
 'B*51:01': 2}

In [31]:
tcr_seq_counts

{'CASSPTLARAKSYEQYF': 3,
 'CATRMAGSSSYEQYF': 2,
 'CASSTGGFGTDTQYF': 4,
 'CATSDQKGTATYEQYF': 2,
 'CASKRGGETQYF': 3,
 'CAWSGTANSPLHF': 2,
 'CSASWGGRTEAFF': 2,
 'CASSQGTSGRNEQFF': 4,
 'CASRTTVSSYEQYF': 2,
 'CAWSVIAGQNTEAFF': 2,
 'CASSLEGVPSYNEQFF': 4,
 'CASSLGDSQETQYF': 2,
 'CASRPGPHGLQPQHF': 4,
 'CASSTGAGGNYGYTF': 3,
 'CASSRNPGQGAKSAYGYTF': 2,
 'CASSFWRDDYGYTF': 5,
 'CAWGTSGGGSYNEQFF': 4,
 'CATSRLAGKGEQYF': 3,
 'CASRRWDANTGELFF': 3,
 'CAISEEGYGELFF': 3,
 'CASSIQGAVGTEAFF': 1,
 'CASTSVSGTYTEAFF': 4,
 'CASRHQGYEQYF': 3,
 'CASRSGLAGGRVFF': 2,
 'CASSLLNSNQPQHF': 2,
 'CASSLDGSSTGELFF': 2,
 'CASSLDPGSYEQYF': 3,
 'CASIFRGPTYEQYF': 2,
 'CASSPAVGNTGELFF': 3,
 'CASSPGQGPRETOQYF': 1,
 'CASSSTSGPAYNEQFF': 2,
 'CASRPLGQKNEKLFF': 3,
 'CARSGVRLAKETQYF': 2,
 'CASSGDRMTLGYTF': 2,
 'CASSARGSGQYF': 1,
 'CASSLGNGYTF': 2,
 'CSAIGQSNEQFF': 1,
 'CATGTGTYEQYF': 1,
 'CASSMTGKDQPQHF': 1,
 'CASSYDSPGTDEKLFF': 1,
 'CAWSKRLAGAYEQYF': 2}

In [32]:
# Calculate the product of Rew_MHC, Rew_TCR, and Classification for each entry
for key, value in selected_seq.items():
    product = value['Rew_MHC'] * value['Rew_TCR'] * value['Classification']
    selected_seq[key]['Product'] = product

# Sort the entries by the product value in descending order and get the top 5
top_5_entries = sorted(selected_seq.items(), key=lambda x: x[1]['Product'], reverse=True)[:5]

# Print top 5 entries
for key, value in top_5_entries:
    print(f"Peptide: {key}, MHC: {value['MHC']}, TCR_Seq: {value['TCR_Seq']}, "
          f"Rew_MHC: {value['Rew_MHC']}, Rew_TCR: {value['Rew_TCR']}, "
          f"Classification: {value['Classification']}, Product: {value['Product']}")

Peptide: KGESIICIIFY, MHC: B*14:02, TCR_Seq: CASRYTTGDSYEQYF, Rew_MHC: 0.998305738949775, Rew_TCR: 0.9857495589472043, Classification: 0.8952227205725236, Product: 0.8809702752051629
Peptide: CCTHCVAYP, MHC: A*25:01, TCR_Seq: CASRAFYGNTIYF, Rew_MHC: 0.9981982499654309, Rew_TCR: 0.9982326198975128, Classification: 0.8827781062770239, Product: 0.8796301674320167
Peptide: SKNPKSRIYCR, MHC: A*02:16, TCR_Seq: CASSLEQGISGYTF, Rew_MHC: 0.9986148254226617, Rew_TCR: 0.9434579002812633, Classification: 0.9110359476899068, Product: 0.8583334714085781
Peptide: FCMTVDYWWWS, MHC: B*15:01, TCR_Seq: CSGTGRSYEQYF, Rew_MHC: 0.9751322700524637, Rew_TCR: 0.9337482878585225, Classification: 0.915806466364606, Product: 0.8338675104280064
Peptide: ARGMCACFYCN, MHC: A*11:01, TCR_Seq: CASSRQSTGELFF, Rew_MHC: 0.9298814764189545, Rew_TCR: 0.9932937592498822, Classification: 0.9017632070417938, Product: 0.8329094988242985
